In [ ]:
! pip install matplotlib

## Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50  # ResNet: Residual Neu. Net
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation
import matplotlib.pyplot as plt 

## Data Generation

In [2]:
batchsize=512  #Batchsize is number of training example used in single iteration

In [3]:
train_data_gen= ImageDataGenerator(rescale=1/255,rotation_range=30,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2, validation_split=0.2)

In [4]:
train_data= train_data_gen.flow_from_directory(r'F:\Drowsiness Detection System\Prepared_Data\train_dataset',
                                target_size=(80,80),batch_size=batchsize,shuffle=True,class_mode='categorical',
                                               subset='training' )

Found 54336 images belonging to 2 classes.


In [5]:
validation_data= train_data_gen.flow_from_directory(r'F:\Drowsiness Detection System\Prepared_Data\train_dataset' ,
                                target_size=(80,80),batch_size=batchsize,shuffle=True,class_mode='categorical', 
                                                    subset='validation')

Found 13583 images belonging to 2 classes.


In [6]:
test_data_gen = ImageDataGenerator(rescale=1./255)

test_data = test_data_gen.flow_from_directory(r'F:\Drowsiness Detection System\Prepared_Data\test_dataset',
                                target_size=(80,80),batch_size=batchsize,shuffle=True,class_mode='categorical')

Found 16979 images belonging to 2 classes.


## Transfer Learning

### Using ResNet50 Model

In [7]:
base_model = ResNet50(include_top=False,weights="imagenet",input_shape=(80,80,3),classes=2)
main_model = base_model.output
main_model = Flatten()(main_model) #to convert into 1-D
main_model = Dense(128, activation='relu')(main_model)
main_model = Dropout(0.5)(main_model)  #to avoid overfitting
main_model = Dense(2,activation= 'softmax')(main_model)  #for classification

model = Model(inputs=base_model.input, outputs= main_model)
for layer in base_model.layers:
    layer.trainable = False       #Filtering out the output layers of ResNet50

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 40, 40, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 20, 20, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 20, 20, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 20, 20, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 20, 20, 256)  1024       ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 10, 10, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 10, 10, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 10, 10, 512)  66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 10, 10, 512)  2048       ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 5, 5, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 5, 5, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 5, 5, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 5, 5, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 5, 5, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 5, 5, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 5, 5, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 3, 3, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 3, 3, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 3, 3, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [10]:
checkpoint = ModelCheckpoint(r'F:\Drowsiness Detection System\Trained_models\resnet_model.h5',
                           monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )
callbacks=[checkpoint,earlystop,learning_rate]

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=8)

C:\Users\chira\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/8
106/106 [==============================] - ETA: 0s - loss: 0.8322 - accuracy: 0.5050 
Epoch 1: val_loss improved from inf to 0.69310, saving model to F:\Drowsiness Detection System\Trained_models\resnet_model.h5
106/106 [==============================] - 3513s 33s/step - loss: 0.8322 - accuracy: 0.5050 - val_loss: 0.6931 - val_accuracy: 0.5061 - lr: 0.0010
Epoch 2/8
106/106 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5058 
Epoch 2: val_loss improved from 0.69310 to 0.69306, saving model to F:\Drowsiness Detection System\Trained_models\resnet_model.h5
106/106 [==============================] - 1858s 18s/step - loss: 0.6931 - accuracy: 0.5058 - val_loss: 0.6931 - val_accuracy: 0.5069 - lr: 0.0010
Epoch 3/8
106/106 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5058 
Epoch 3: val_loss improved from 0.69306 to 0.69306, saving model to F:\Drowsiness Detection System\Trained_models\resnet_model.h5
106/106 [=======================

In [13]:
loss_tr, acc_tr = model.evaluate_generator(train_data)
print(loss_tr)
print(acc_tr)

C:\Users\chira\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


0.6930769085884094
0.5059260725975037


In [14]:
loss_vd, acc_vd = model.evaluate_generator(validation_data)
print(loss_vd)
print(acc_vd)

C:\Users\chira\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


0.6930768489837646
0.5059265494346619


In [15]:
loss_test, acc_test = model.evaluate_generator(test_data)
print(loss_test)
print(acc_test)

C:\Users\chira\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


0.6930769681930542
0.5059190988540649


### Using VGG16 Model

In [2]:
base_modelvgg = VGG16(include_top=False,weights="imagenet",input_shape=(80,80,3),classes=2)
main_modelvgg = base_modelvgg.output
main_modelvgg = Flatten()(main_modelvgg) #to convert into 1-D
main_modelvgg = Dense(128, activation='relu')(main_modelvgg)
main_modelvgg = Dropout(0.5)(main_modelvgg)  #to avoid overfitting
main_modelvgg = Dense(2,activation= 'softmax')(main_modelvgg)  #for classification

modelvgg = Model(inputs=base_modelvgg.input, outputs= main_modelvgg)
for layer in base_modelvgg.layers:
    layer.trainable = False       #Filtering out the output layers of VGG16

NameError: name 'VGG16' is not defined

In [22]:
modelvgg.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 80, 80, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 80, 80, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 80, 80, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 40, 40, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 40, 40, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 40, 40, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 20, 20, 128)       0   

In [23]:
checkpointvgg = ModelCheckpoint(r'F:\Drowsiness Detection System\Trained_models\vgg_model.h5',
                           monitor='val_loss',save_best_only=True,verbose=3)

In [1]:
modelvgg.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

modelvgg.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=8)

NameError: name 'modelvgg' is not defined